# Gram Schmidt Process - Examples, code, Exercises

A great reference for this is Examples 6.33 and 6.58 in Sheldon Axler's <a href="http://linear.axler.net"><em>Linear Algebra Done Right</em></a>, from <a href="http://linear.axler.net/InnerProduct.pdf">the chapter on Inner Product Spaces</a>.

In [1]:
import numpy as np

## In $\mathbb{R}^n$
First, let's write a simple program that implements the Gram-Schmidt process for a list of vectors in $\mathbb{R}^n$ using the ordinary dot product for the inner product.

In [2]:
def gram_schmidt(M):
    # M is a list of vectors
    k=len(M)
    E=[] # initialize the output
    for i in range(k):
        fi=M[i] # fi is the i-th vector of M
        for j in range(i): # This loop only exists when i>0
            fi+=-np.dot(E[j],M[i])*E[j]
        E.append(1/np.sqrt(np.dot(fi,fi))*fi)
    return E

In [3]:
myvecs=np.array([[1.0,2,3],[1.0,1,1],[0,1,0]])
print myvecs

[[ 1.  2.  3.]
 [ 1.  1.  1.]
 [ 0.  1.  0.]]


In [4]:
mye=gram_schmidt(myvecs)

In [5]:
print mye[0]
print mye[1]
print mye[2]

[ 0.26726124  0.53452248  0.80178373]
[ 0.87287156  0.21821789 -0.43643578]
[-0.40824829  0.81649658 -0.40824829]


Look, the results are orthonormal, up to many significant digits

In [6]:
for i in range(3):
    for j in range(3):
        print "i=",i,"j=",j," <ei,ej>=",np.dot(mye[i],mye[j])

i= 0 j= 0  <ei,ej>= 1.0
i= 0 j= 1  <ei,ej>= -1.66533453694e-16
i= 0 j= 2  <ei,ej>= 0.0
i= 1 j= 0  <ei,ej>= -1.66533453694e-16
i= 1 j= 1  <ei,ej>= 1.0
i= 1 j= 2  <ei,ej>= -2.77555756156e-17
i= 2 j= 0  <ei,ej>= 0.0
i= 2 j= 1  <ei,ej>= -2.77555756156e-17
i= 2 j= 2  <ei,ej>= 1.0


## Example using polynomials and a different inner product

In example 6.33 of Axler's book, he consders the vector space $V$ of continuous functions on $[-1,1]$ and defines the inner product by
$$\langle f, g\rangle = \int_{-1}^{1} f(x)g(x) \,dx.$$
Let's import sympy and declare $x$ as a variable so we can easily define and manipulate functions.

In [2]:
from sympy import *

In [3]:
x = Symbol('x')

In [10]:
import scipy.integrate as integrate

In [4]:
from math import pi # we want this later

In [5]:
pi

3.141592653589793

In [20]:
def ip2(f,g):
    return integrate(f*g,(x,-1,1))

"Integrate" is a function from sympy that we can drop in and use for this demonstration.  In an enterprise application, you'd probably be better off doing numerical integration, something like quad from scipy.integrate.

In [21]:
ip2(sin(x),exp(x))

-E*cos(1)/2 + exp(-1)*cos(1)/2 + exp(-1)*sin(1)/2 + E*sin(1)/2

Now, we can find an orhtonormal basis for the space of polynomials of degree $\leq 2$.

In [22]:
k=2 # max degree
myE=[] # initialize the output with zeros
for i in range(k+1):
    fi=x**i
    for j in range(i):
        fi+=-ip2(myE[j],x**i)*myE[j]
    myE.append(fi/(sqrt(ip2(fi,fi))))

In [24]:
for p in myE:
    print p

sqrt(2)/2
sqrt(6)*x/2
3*sqrt(10)*(x**2 - 1/3)/4


Compare our results to Axler's example.

## Now, example 6.58
Find a polynomial $u$ with real coefficients and degree at most $5$ that approximates $\sin(x)$ as well as possible on the interval $[-\pi,\pi]$
the sense that 

$$\int_{-\pi}^{\pi} |\sin(x)-u(x)|^2 \, dx$$

is as small as possible. Compare this result to the Taylor series approximation.

The criterion stated for approximating ``as well as possible'' means that $u(x)$ is the polynomial closest to $\sin(x)$, using the norm defined by the innter product

$$\langle f,g \rangle = \int_{-\pi}^{\pi}f(x)g(x)\,dx.$$

We can find $u$ by othogonal projection of $\sin(x)$ on the subspace $U$ of polynomials of degree $\leq 5.$  For this, we first find an orthonormal basis for $U$.

We have to change the limits of integration, but we'll make another change too.  We'll introduce a numerical integration.

In [82]:
import scipy.integrate as integrate

In [83]:
def ip3(f,g):
    h=f*g;
    return integrate.quad(lambda t: h.subs(x,t),-pi,pi)[0]

In [84]:
ip3(sin(x),x**3)

24.313441517522115

In [85]:
k=5 # max degree
myE=[] # initialize the output with zeros
for i in range(k+1):
    fi=x**i
    for j in range(i):
        fi+=-ip3(myE[j],x**i)*myE[j]
    myE.append(fi/(sqrt(ip3(fi,fi))))

In [86]:
for p in myE:
    print p

0.398942280401433
0.219948406790773*x
0.135577175410079*x**2 - 0.446031029038193
0.0851039026947899*x**3 - 0.503965111551828*x
0.0537538886225829*x**4 - 0.454739670621524*x**2 + 0.448810065451612
0.0340492544849719*x**5 - 0.373391857687433*x**3 + 0.789692126134889*x


Now, we orthogonally project $\sin(x)$ onto $U$.

In [89]:
u=0
for p in myE:
    u+= ip3(sin(x),p) * p
print u

0.00564311797634686*x**5 - 0.155271410633429*x**3 + 0.987862135574676*x


Here's a picture:  the sine is in red, the approximation $u$ is in blue.  The fifth degree Taylor polynomial is in green.

In [93]:
myplot=plot(sin(x),u,x-x**3 /6.+x**5 / 120.0,(x,-4,4),ylim=[-1,1],show=False)
myplot[0].line_color = 'r'
myplot[2].line_color = 'g'
myplot.show()

### One more inner product

In [132]:
np.linspace(-3,3,6)

array([-3. , -1.8, -0.6,  0.6,  1.8,  3. ])

In [138]:
def ip3(f,g):
    xvals=np.linspace(-3,3,6)
    out=0
    for xval in xvals:
        out+=f.subs(x,xval)*g.subs(x,xval)
    return out

In [139]:
ip3(sin(x),x**3)

19.2233647503028

In [140]:
k=5 # max degree
myF=[] # initialize the output with zeros
for i in range(k+1):
    fi=x**i
    for j in range(i):
        fi+=-ip3(myF[j],x**i)*myF[j]
    myF.append(fi/(sqrt(ip3(fi,fi))))

In [141]:
for p in myF:
    print p

sqrt(6)/6
0.199204768222399*x + 2.21161720295625e-17
0.113655151164579*x**2 + 2.01089350016141e-17*x - 0.477351634891233
0.0718900455729099*x**3 - 0.522784411406201*x + 5.81045802328965e-18
0.0531634056525003*x**4 - 0.519482420947289*x**2 - 9.19117007040307e-17*x + 0.558088167177687
0.0531634056525003*x**5 + 3.81565757201043e-16*x**4 - 0.616695505569003*x**3 - 3.72844254179305e-15*x**2 + 1.26502171854679*x + 3.59823467707938e-15


In [142]:
v=0
for p in myF:
    v+= ip3(sin(x),p) * p
print v

0.00615078036331832*x**5 + 1.66628436800185e-17*x**4 - 0.161047089652637*x**3 - 1.51778812318291e-16*x**2 + 0.998250600131574*x + 4.42956011211322e-17


Here are a few closeups

In [146]:
myplot=plot(sin(x),u,v,(x,.5,.7),ylim=[.5,.7],show=False)
myplot[0].line_color = 'r'
myplot[2].line_color = 'g'
myplot.show()

In [150]:
myplot=plot(sin(x),u,v,(x,1.2,1.4),ylim=[.9,1.05],show=False)
myplot[0].line_color = 'r'
myplot[2].line_color = 'g'
myplot.show()

# Project: Approximate the sine in six ways.  

Find a cubic polynomial $u$ that best approximates $\sin(t)$ in the sense that $\|u(t)-\sin(t)\|$ is minimized.  The definition of the norm $\| \qquad \|$, however, depends on a choice of inner product.  Here are six different ways to define inner products on the vector space of polynomials of degree less than or equal to $3$.

\begin{align*}
\langle f,g\rangle_6&=f(3)g(3)+f'(3)g'(3)+f''(3)g''(3)+f'''(3)g'''(3)\\
 \langle f,g\rangle_5&=\int_0^6 f(t)g(t)dt \\
 \langle f,g\rangle_3&=f(0)g(0)+f(2)g(2)+f(4)g(4)+f(6)g(6)\\
\langle f,g\rangle_2&=f(1)g(1)+f'(1)g'(1)+f(5)g(5)+f'(5)g'(5)\\
 \langle f,g\rangle_4&=f(0)g(0)+f(3)g(3)+f'(3)g'(3)+f(6)g(6)\\
 \langle f,g\rangle_1&=f(0)g(0)+f'(0)g'(0)+f(6)g(6)+f'(6)g'(6)
\end{align*}


I encourage you to work in groups and turn in one assignment per group.  Write your solutions in a seperate Jupyter notebook.  Write clearly documented code, create some good pictures, and write observations and conclusions.